# Model Structure

**Para-disk** is designed to produce model images of emission from molecular gas and/or dust in the disk around a young star. This calculation is performed assuming a parametric structure for the temperature, density, etc. Below we outline this parametric structure, and some of the assumptions that go into it. 

:::{important}

Within this code there are a couple of different options for the parametric structure. You may choose a different structure based on the type of system that you are modeling (e.g., protoplanetary disk vs. debris disk) or the complexity of the data (e.g., data with low spatial resolution data might warrant a simpler disk structure).

You choose different structure by specifying the version of disk.py that you use. The options are:
- **disk.py**: Protoplanetary disk structure, with a surface density that follows a power law with an exponential tail, and a vertical temperature gradient.
- **disk_pow.py**: Same as above, but the surface density only follows a power law, with no exponential tail at large radii.
- **disk_ecc.py**: Same as disk.py, but with an eccentric disk. This accounts for both changes in density and velocity with azimuth in the disk.
- **debris_disk.py**: A power law surface density, with a gas temperature that does not depend on height in the disk and is calculated using the equilibrium temperature of a blackbody grain. Instead of calculating the vertical density profile based on hydrostatic equilibrium, this code uses a specified value of the gas scale height. Intended for modeling dusty debris disks.
:::

## Disk Structure

### disk.py: Protoplanetary Disk

This model uses parametric forms of the temperature and density structure that are meant to represent protoplanetary disks, with a vertical temperature gradient, and a self-similar surface density distribution. 

The temperature follows a power law with radius, with a vertical temperature gradient connecting the cold midplane with the warm atmosphere:

$$T_{atm} = T_{atm0} \left(\frac{r}{150 au}\right)^{q}$$
$$T_{mid} = T_{mid0} \left(\frac{r}{150 au}\right)^{q}$$
$$T_{gas} = \begin{cases} T_{atm}+(T_{mid}-T_{atm})\left(cos\frac{\pi z}{2 Z_q}\right)^{2} \; \; \; \; &z<Z_q\\
                            T_{atm} \; \; \; \; &z>Z_q \end{cases}$$
$$Z_q = Z_{q0} \left(\frac{r}{150 au}\right)^{1.3}$$

The parameter $Z_q$ is the height above the midplane at which the gas temperature reaches its maximum value. Common values for $Z_{q0}$ might be a fixed number (e.g., 70 au, Rosenfeld et al. 2013) or a multiple of the pressure scale height (e.g., Dartois et al. 2003). 

The surface density follows a power law with an exponential tail, as expected for a viscously evolving disk (e.g., Lynden-Bell & Pringle 1974, Hartmann et al. 1998):

$$\Sigma_{gas}(r) = \frac{M_{gas}(2-\gamma)}{2 \pi R_c^2} \left(\frac{r}{R_c}\right)^{-\gamma}\exp\left[-\left(\frac{r}{R_c}\right)^{2-\gamma}\right]$$

where $M_{gas}$, $R_c$, and $\gamma$ are the gas mass, critical radius, and power law index. 

Once the surface density and temperature have been specified, the volume density is calculated using hydrostatic equilibrium:

$$-\frac{\partial\ln\rho}{\partial z} = \frac{\partial\ln T}{\partial z} = \frac{1}{c_s^2}\left[\frac{GM_*z}{(r^2+z^2)^{3/2}}\right]$$

$$c_s^2 = \frac{k_B T}{\mu m_h}$$

where $\mu$ is the mean molecular weight (=2.37). 


The velocity profile is Keplerian motion, with corrections for the height above the midplane and the pressure gradient.

$$\frac{v_{\phi}^2}{r} = \frac{GM_*r}{(r^2+z^2)^{3/2}}+\frac{1}{\rho_{gas}}\frac{\partial P_{gas}}{\partial r}$$

The line profile is assumed to be a Gaussian whose width ($\Delta V$) is set by the thermal and non-thermal motion. The non-thermal motion ($\delta v_{turb}$) can be taken as proportional to the local sound speed:

$$\Delta V = \sqrt{(2k_B T(r,z)/m_{CO})(1+\delta v_{turb}^2)}$$

or as a fixed velocity:

$$\Delta V = \sqrt{(2k_B T(r,z)/m_{CO})+\delta v_{turb}^2}$$


### disk_pow.py: Power-law protoplanetary disk

This model is identical to above, except that the surface density is assumed to follow a power law, without an exponential tail:

$$\Sigma_{gas} = \frac{M_{gas}(2-\gamma)}{2\pi (R_{out}^{2-\gamma}-R_{in}^{2-\gamma})}r^{-\gamma}$$



### disk_ecc.py: Eccentric protoplanetary disk

This model is similar to the protoplanetary disk model in disk.py, but it includes a prescription for eccentricity in the disk.

One modification comes in the surface density structure. The disk is treated as a series of concentric elliptical rings of semi-major axis $a$. The surface density is the same tapered power law as in disk.py, with the semi-major axis in place of the radial distance from the central star.

$$\Sigma_{gas}(a) = \frac{M_{gas}(2-\gamma)}{R_c^2} \left(\frac{a}{R_c}\right)^{-\gamma}\exp\left[-\left(\frac{a}{R_c}\right)^{2-\gamma}\right]$$

The surface density also needs a dependence on the angle from periapsis, $\phi$, since there will be a higher density near apastron, where the material spends most of its orbit, than at periastron, where the material spends less of its orbit. The density per unit length around a given orbit, $\lambda$, is given by

$$\lambda = \frac{m\sqrt{1-e^2}}{2\pi a(1+e\cos\phi)}$$

where $m$ is the mass contained within the ring. The linear density of the $i$th ring is related to the surface density through:

$$\Sigma(\phi) = \frac{\lambda_i(\phi)}{(r_{i+1}(\phi)-r_{i-1}(\phi))/2}$$

where $r$ is the distance from the central star (at one of the foci of the ellipse) as a function of semi-major axis and angle from periapsis.

$$r_i(\phi) = \frac{a_i(1-e^2)}{1+e\cos\phi}$$

The result is a surface density that depends on both semi-major axis and angle from periapsis:

$$\Sigma_{gas} = \frac{\Sigma_{gas}(a)(1-e^2)^{3/2}}{2\pi (1+e\cos\phi)^2}$$

In addition, the velocity profile is modified to account for the variations with $\phi$ (see derivation [here](https://pdfs.semanticscholar.org/75d1/c8533025d0a7c42d64a7fef87b0d96aba47e.pdf)).

$$v = \sqrt{\frac{GM_*}{a(1-e^2)}}\cos(\omega+\phi)+e\cos\omega$$

where $\omega$ in the angle of periastron (= the angle between periastron and the major axis). When multiplied by $\sin i$ this becomes the projected velocity along the line of sight. *Note that this does not include the correction for the height above the midplane, or for the pressure gradient.*


### debris_disk.py: Debris disk structure

This model is intended to predict the dust emission from a debris disk. It calculates the dust temperature assuming blackbody dust grains, based on the stellar luminosity and it assumes that the vertical density distribution has a Gaussian shape, with the scale height taken as a free parameter ($h = H/R$, or $h=H$, where $h$ is one of the model inputs). The surface density is a power law with radius (*note that the sign on $p$ is different than in the previous models*). This model only includes dust, and does not model the gas. 

$$T_d = \left( \frac{L_*}{16 \pi d^2 \sigma_B} \right)^{1/4}$$

$$\Sigma = \begin{cases} \Sigma_c r^p \; \; \; \; &r_i < r < r_o \\
                        0            \; \; \; \; &\rm{otherwise} \end{cases}$$
                        
$$\Sigma_c = \frac{M_{dust}(2+p)}{2\pi(R_{out}^{2+p}-R_{in}^{2+p})} $$

$$\rho_{dust} = \frac{\Sigma}{H\sqrt{\pi}}\exp\left(-\left(\frac{z}{H}\right)^2\right)$$

### Self-gravity

The codes disk.py and disk_pow.py can also include the self-gravity of the disk as a modification on the orbital velocity. This follows the prescription in Lodato 2007, Bertin & Lodato 1999 (used by Verosini et al. 2021 to model Elias 2-27). In the presence of self-gravity the orbital frequency becomes:
  
  $$\frac{v_{\phi}^2}{r} = \frac{GM_*r}{(r^2+z^2)^{3/2}}+\frac{1}{\rho_{gas}}\frac{\partial P_{gas}}{\partial r}+\frac{\partial \phi_{gas}}{\partial r}$$
  
  where $\phi_{gas}$ is the potential due to the self-gravity of the disk. This code does *not* include the modification to the vertical hydrostatic equilibrium due to self-gravity (e.g. eqn 18 of Rosenfeld et al. 2013). It only includes the modification to the velocity field. The gradient in the potential is calculated using:
  
  $$\frac{\partial \phi_{gas}}{\partial r}(r,z) = \frac{G}{r}\int^{\inf}_{0}\left[K(k)-\frac{1}{4}\left(\frac{k^2}{1-k^2}\right)\left(\frac{R}{r}-\frac{r}{R}+\frac{z}{rR}\right)E(k)\right]\sqrt{\frac{R}{r}}k\Sigma (R)dR$$
  
  where $k^2=4Rr/[(r+R)^2+z^2]$ and $E$ and $K$ are complete elliptic integrals of the first kind. When using self-gravity, you are encouraged to increase Rout to improve the accuracy of the integral above, especially if you are interested in the velocities near the outer edge of the disk. [*Only available in disk.py and disk_pow.py*]

## Radiative Transfer

To determine the emission, the radiative transfer equation must be integrated along each line of sight through the disk:

$$ I_{\nu} = \int^{\infty}_0 S_{\nu}(s)\exp[-\tau_{\nu}(s)]K_{\nu}(s)ds $$

where $\tau_{\nu}(s) = \int_0^s K_{\nu}(s')ds'$ is the optical depth, $K_{\nu}$ is the absorption coefficient, and $S_{\nu}$ is the source function. 